## HW 4 Part 2A: We Have Time!

In this part of the homework, we're going to explore what happens when we have **time series data** – sequential data that is indexed to time. Do our standard regression tools still work? If not, how can we modify them such that they do?

**NOTE**: This part of the homework requires **no coding**. The only questions you need to answer are written questions based on what you observe in the code. That said, feel free to read through the code and see if you can understand it. If not, no worries!

In [ ]:
# JUST RUN THIS CELL - Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

### River Flow Rate

Let's load in the dataset `river_flow_rate.csv`

In [ ]:
# JUST RUN THIS CELL
river_flow_rate = pd.read_csv("river_flow_rate.csv")
river_flow_rate

As you can see, we have two columns:
- `time`, which reflects the time index at which we measured the data
- `river_flow_rate`, which measures how fast the river was flowing in cubic meters per second (CMS) at each timestep

Let's plot the data and see what it looks like:

In [ ]:
# JUST RUN THIS CELL
plt.plot(river_flow_rate["time"], river_flow_rate["river_flow_rate"])
plt.xlabel("Timestep")
plt.ylabel("River Flow Rate (CMS)")
plt.title("Time Series Data of River Flow Rate in Cubic Meters per Second (CMS)")
plt.show()

### 1.1 - Make Your Own Guess
Let's say you wanted to forecast the river flow in the next timestep. Based on the historic trend of the time series in the last few timesteps, do you expect the river flow rate in the next timestep to go up or down (or, which one is more likely)? What about the timestep after that? Is there a pattern to the time series?

(YOUR ANSWER GOES HERE.)

### 1.2 - Your Friend's Suggestion

Your friend, who likes to mansplain a lot, tells you that you should just fit a linear regression model to the data and it will make all the forecasts you need. Do you think a linear regression would be good for this type of data? Why or why not?

(YOUR ANSWER GOES HERE.)

For now, we will take your friend's suggestion and see what happens if we try fitting a linear regression to the data and make forecasts.

- **Note:** If you examine the code, you will notice we did **not** make a **random** train-test split like we normally do. In time series data, since all the observations are in sequential order, each observation depends on the previous observations. So, we use a **time-based train-test split** where we just split the sequential data in the order it was given to us. Other than that, our common formula of **split, fit, predict** still works.
- **Note:** We will use Mean Absolute Error (MAE) as our evaluation metric, but it is common to see other evaluation metrics in time series forecasting, as well.

In [ ]:
# JUST RUN THIS CELL - Fitting the regression model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Time based train test split!
split_index = int(len(river_flow_rate) * 0.8)
train_data = river_flow_rate.iloc[:split_index]
test_data = river_flow_rate.iloc[split_index:]

# Preparing the data
X_train = train_data[["time"]]
y_train = train_data["river_flow_rate"]
X_test = test_data[["time"]]
y_test = test_data["river_flow_rate"]

# Fit a Linear Regression model
model = LinearRegression(fit_intercept=False)
model.fit(X_train, y_train)

# Make predictions
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Store predictions in the test DataFrame
test_data = test_data.copy()
test_data["predicted_flow_rate"] = y_pred_test

# Forecast future values (time steps 200 onwards)
future_times = np.arange(200, 220)  # Predicting 20 future time steps
future_X = future_times.reshape(-1, 1)  # Reshape for model input
future_predictions = model.predict(future_X)  # Forecast future values

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred_test)

# Plot training predictions, testing predictions, and forecasted values
plt.figure(figsize=(10, 6))
plt.plot(river_flow_rate["time"], river_flow_rate["river_flow_rate"], label="Actual River Flow Rate")
plt.plot(train_data["time"], y_pred_train, label="Training Predictions", linestyle="dashed")
plt.plot(test_data["time"], y_pred_test, label="Testing Predictions", linestyle="dashed")
plt.plot(future_times, future_predictions, label="Forecasted Values", linestyle="dashed")

# Labels and legend
plt.xlabel("Time")
plt.ylabel("River Flow Rate")
plt.title(f"Linear Regression Forecasting on Time Series Data\nMAE: {mae:.2f}")
plt.legend()
plt.grid(True)
plt.show()

### 1.3 - Assessing the Linear Regression

Do you think that the linear regression model was a good fit for the time series data? Do you think its forecasts are reliable? Why or why not?

(YOUR ANSWER GOES HERE.)

When we have time series data, linear regression models are pretty good at giving us a general sense of the **signal** or **trend** of the data. They can roughly tell us whether a time series is increasing, decreasing, or staying relatively constant over time.

However, in addition to signal, time series also have **cyclical** or **seasonal fluctuations** that our linear models are just not really well-equipped to handle.

One thing about time series data is that since all the observations are indexed to time, in a lot of cases, the best predictor of what happens today is what happened yesterday. If a time series alternates between going up and down at every timestep, then if the time series was up yesterday, we can say with reasonable certainty that the time series will be down today.

This concept is called **autocorrelation** - a time series is correlated with itself because its past values can predict future values. While we may not be able to generate forecasts by regressing the time series on its own timesteps, autocorrelation does allow us to do one thing: **we can regress the time series on past values of itself!**

This is called **autoregression**. In autoregressive (AR) models, we regress a time series on a **lagged version** of itself. The number of lags we use (often called $p$) is the *lag order* of the model, and we say this defines an AR(p) model or process.

### 1.4 - Autoregression with Lagged Features

Let's see what happens when we use the values of `river_flow_rate` from 1 timestep before to predict the current value (and eventually make future forecasts).

In [ ]:
# JUST RUN THIS CELL - Adding a lag-1 feature

ar1 = river_flow_rate.copy()
ar1["lag1"] = ar1["river_flow_rate"].shift(1)
ar1

In [ ]:
# JUST RUN THIS CELL - A scatterplot of river flow rate vs. a one-timestep lagged version of itself
autocorr_coef = np.corrcoef(ar1["lag1"].iloc[1:], ar1["river_flow_rate"].iloc[1:])[0][1]
plt.scatter(ar1["lag1"], ar1["river_flow_rate"])
plt.xlabel("River Flow Rate At Time t-1")
plt.ylabel("River Flow Rate At Time t")
plt.title(f"Autocorrelation Between River Flow Rate and its Lag 1 Values \n(r = {autocorr_coef})")
plt.show()

In [ ]:
# JUST RUN THIS CELL - Making AR(1) Forecasts

# Time based train test split!
ar1_nona = ar1.dropna()
split_index = int(len(ar1_nona) * 0.8)
train_data = ar1_nona.iloc[:split_index]
test_data = ar1_nona.iloc[split_index:]

# Preparing the data
X_train = train_data[["lag1"]]
y_train = train_data["river_flow_rate"]
X_test = test_data[["lag1"]]
y_test = test_data["river_flow_rate"]

# Fit a Linear Regression model
model = LinearRegression(fit_intercept=False)
model.fit(X_train, y_train)

# Make predictions
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Store predictions in the test DataFrame
test_data = test_data.copy()
test_data["predicted_flow_rate"] = y_pred_test

# Iterative forecasting for future time steps (200 onwards)
future_times = np.arange(200, 220)  # Predicting 20 future time steps
future_forecasts = []

# Initialize with the last observed lagged value from the test set
last_observed_value = test_data["lag1"].iloc[-1]

# Iteratively generate future forecasts
for _ in range(len(future_times)):
    next_prediction = model.predict([[last_observed_value]])[0]
    future_forecasts.append(next_prediction)
    last_observed_value = next_prediction

future_forecasts = np.array(future_forecasts)

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred_test)

# Plot training predictions, testing predictions, and forecasted values
plt.figure(figsize=(10, 6))
plt.plot(ar1_nona["time"], ar1_nona["river_flow_rate"], label="Actual River Flow Rate")
plt.plot(train_data["time"], y_pred_train, label="Training Predictions", linestyle="dashed")
plt.plot(test_data["time"], y_pred_test, label="Testing Predictions", linestyle="dashed")
plt.plot(future_times, future_forecasts, label="Forecasted Values", linestyle="dashed")

# Labels and legend
plt.xlabel("Time")
plt.ylabel("River Flow Rate")
plt.title(f"AR(1) Linear Regression Forecasting on Time Series Data\nMAE: {mae:.2f}")
plt.legend()
plt.grid(True)
plt.show()

#### Did our model do better or worse at forecasting than the linear regression model? Did you expect the results to be better? How did the model do at forecasting within sample (test set) vs. out of sample (forecasts)?

(YOUR ANSWER GOES HERE.)

### 1.5 - AR(2) Model

Instead of using just 1 lag, let's see what happens if we use 2 lags. This would be an AR(2) model.

In [ ]:
# JUST RUN THIS CELL - Adding a lag-2 feature

ar2 = ar1.copy()
ar2["lag2"] = ar2["lag1"].shift(1)
ar2

In [ ]:
# JUST RUN THIS CELL - A scatterplot of river flow rate vs. a two-timestep lagged version of itself
autocorr_coef = np.corrcoef(ar2["lag2"].iloc[2:], ar2["river_flow_rate"].iloc[2:])[0][1]
plt.scatter(ar2["lag2"], ar2["river_flow_rate"])
plt.xlabel("River Flow Rate At Time t-2")
plt.ylabel("River Flow Rate At Time t")
plt.title(f"Autocorrelation Between River Flow Rate and its Lag 2 Values \n(r = {autocorr_coef})")
plt.show()

In [ ]:
# JUST RUN THIS CELL - Making AR(2) Forecasts

# Time based train test split!
ar2_nona = ar2.dropna()
split_index = int(len(ar2_nona) * 0.8)
train_data = ar2_nona.iloc[:split_index]
test_data = ar2_nona.iloc[split_index:]

# Preparing the data
X_train = train_data[["lag1", "lag2"]]
y_train = train_data["river_flow_rate"]
X_test = test_data[["lag1", "lag2"]]
y_test = test_data["river_flow_rate"]

# Fit a Linear Regression model
model = LinearRegression(fit_intercept=False)
model.fit(X_train, y_train)

# Make predictions
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Store predictions in the test DataFrame
test_data = test_data.copy()
test_data["predicted_flow_rate"] = y_pred_test

# Iterative forecasting for future time steps (200 onwards)
future_times = np.arange(200, 220)  # Predicting 20 future time steps
future_forecasts = []

# Initialize with the last two observed lagged values from the test set
last_lag1 = test_data["lag1"].iloc[-1]
last_lag2 = test_data["lag2"].iloc[-1]

# Iteratively generate future forecasts for AR(2)
for _ in range(len(future_times)):
    next_prediction = model.predict([[last_lag1, last_lag2]])[0]  # Use both lag1 and lag2
    future_forecasts.append(next_prediction)
    last_lag2 = last_lag1  # Shift lag1 to lag2
    last_lag1 = next_prediction  # Use new prediction as lag1

future_forecasts = np.array(future_forecasts)

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred_test)

# Plot training predictions, testing predictions, and forecasted values
plt.figure(figsize=(10, 6))
plt.plot(ar2_nona["time"], ar2_nona["river_flow_rate"], label="Actual River Flow Rate")
plt.plot(train_data["time"], y_pred_train, label="Training Predictions", linestyle="dashed")
plt.plot(test_data["time"], y_pred_test, label="Testing Predictions", linestyle="dashed")
plt.plot(future_times, future_forecasts, label="AR(2) Forecasted Values", linestyle="dashed")

# Labels and legend
plt.xlabel("Time")
plt.ylabel("River Flow Rate")
plt.title(f"AR(2) Linear Regression Forecasting on Time Series Data\nMAE: {mae:.2f}")
plt.legend()
plt.grid(True)

# Show the plot
plt.show()

#### Did adding a second lag improve or worsen our model? How do you think we could make our model better? What do you notice about the out-of-sample forecasts vs. the test set predictions? Why do you think that is? (Just a guess, doesn't have to be correct.)

(YOUR ANSWER GOES HERE.)

### 1.6 - A Spectral Model

One last option we have is instead of regressing our time series data on past values of itself, we can instead regress it on **sine and cosine features** created based on the periodicity of the time series. In time series forecasting, this is called **spectral analysis**, since we are viewing the time series problem more as a sinusoidal signal to fit to rather than a stochastic process for the purposes of modeling.

Representing a time series with a basis of various sine and cosine functions is called a **Fourier decomposition**, and it drives many of the industry-standard forecasting tools like Meta's Prophet.

Run the cell below to see what happens to our forecast when we use a Fourier decomposition to make forecasts.

In [ ]:
# JUST RUN THIS CELL - Fourier Decomposition

# Define the number of Fourier components to use
num_fourier_terms = 30
season_length = 1

# Time-based train-test split (80% train, 20% test)
split_index = int(len(river_flow_rate) * 0.8)
train_data = river_flow_rate.iloc[:split_index]
test_data = river_flow_rate.iloc[split_index:]

# Generate Fourier basis functions for train and test sets
time_train = train_data["time"].values.reshape(-1, 1)
time_test = test_data["time"].values.reshape(-1, 1)

fourier_features_train = np.hstack([
    np.sin(2 * np.pi * (i + 1) * time_train / season_length) for i in range(num_fourier_terms)
] + [
    np.cos(2 * np.pi * (i + 1) * time_train / season_length) for i in range(num_fourier_terms)
])

fourier_features_test = np.hstack([
    np.sin(2 * np.pi * (i + 1) * time_test / season_length) for i in range(num_fourier_terms)
] + [
    np.cos(2 * np.pi * (i + 1) * time_test / season_length) for i in range(num_fourier_terms)
])

# Fit a Linear Regression model using Fourier features on training data
fourier_model = LinearRegression(fit_intercept=False)
fourier_model.fit(fourier_features_train, train_data["river_flow_rate"])

# Make predictions on train and test sets
y_pred_train = fourier_model.predict(fourier_features_train)
y_pred_test = fourier_model.predict(fourier_features_test)

# Store predictions in the test DataFrame
test_data = test_data.copy()
test_data["predicted_flow_rate"] = y_pred_test

# Iterative forecasting for future time steps (200 onwards)
future_times = np.arange(200, 220).reshape(-1, 1)

future_fourier_features = np.hstack([
    np.sin(2 * np.pi * (i + 1) * future_times / season_length) for i in range(num_fourier_terms)
] + [
    np.cos(2 * np.pi * (i + 1) * future_times / season_length) for i in range(num_fourier_terms)
])

# Predict future values using the Fourier model
future_forecasts = fourier_model.predict(future_fourier_features)

# Calculate MAE for Fourier model on the test set
mae_fourier = mean_absolute_error(test_data["river_flow_rate"], y_pred_test)

# Plot training predictions, testing predictions, and forecasted values
plt.figure(figsize=(10, 6))

# Plot actual river flow rate
plt.plot(river_flow_rate["time"], river_flow_rate["river_flow_rate"], label="Actual River Flow Rate")

# Plot Fourier model predictions on training set
plt.plot(train_data["time"], y_pred_train, label="Training Predictions", linestyle="dashed")

# Plot Fourier model predictions on test set
plt.plot(test_data["time"], y_pred_test, label="Testing Predictions", linestyle="dashed")

# Plot Fourier model future forecasts
plt.plot(future_times, future_forecasts, label="Fourier Forecasted Values", linestyle="dashed")

# Labels and legend
plt.xlabel("Time")
plt.ylabel("River Flow Rate")
plt.title(f"Fourier Series Regression Forecasting on Time Series Data\nMAE: {mae_fourier:.2f}")
plt.legend()
plt.grid(True)

# Show the plot
plt.show()

#### How does the Test Set MAE for our Fourier Model compare to the Test Set MAEs of the AR(p) models and our naive regression? How do our forecasted values compare? Among all the forecasted values we've generated, which model's forecasts do you believe the most? Does your answer surprise you? Does it follow the mantra of "a good model is the one that has the highest prediction accuracy on unseen data?"

(YOUR ANSWER GOES HERE.)